In [1]:
# Need this for type mapping
from google.protobuf.descriptor_pb2 import FieldDescriptorProto
# And this is the compiled proto output
import ws_messages_pb2
# For string templating
from io import StringIO
import graphviz

In [2]:
# a mapping with values such as 1: 'double', 9: 'string', etc. to find the text value of a type
type_mapping = {number: text.lower().replace("type_", "") for text, number in FieldDescriptorProto.Type.items()}

# our compiled type actually includes .DESCRIPTOR where we can find instrospection data
types = ws_messages_pb2.DESCRIPTOR.message_types_by_name

for _type, message in types.items():
    print(f"type: [{_type}]")
    for _field in message.fields:
        message_type = _field.message_type
        if message_type:
            message_type = message_type.name
        print(f"  field: [{_field.name}] of type [{type_mapping[_field.type]}], and message type is [{message_type}]")

type: [PbMeta]
  field: [title] of type [string], and message type is [None]
  field: [description] of type [string], and message type is [None]
  field: [URL] of type [string], and message type is [None]
  field: [user_defined] of type [string], and message type is [None]
type: [PbTimeZone]
  field: [hours] of type [int32], and message type is [None]
  field: [minutes] of type [int32], and message type is [None]
  field: [string_basic] of type [string], and message type is [None]
  field: [string_extended] of type [string], and message type is [None]
type: [PbStateTotals]
  field: [runahead] of type [int32], and message type is [None]
  field: [waiting] of type [int32], and message type is [None]
  field: [held] of type [int32], and message type is [None]
  field: [queued] of type [int32], and message type is [None]
  field: [expired] of type [int32], and message type is [None]
  field: [ready] of type [int32], and message type is [None]
  field: [submit_failed] of type [int32], and m

In [3]:
# This dictionary is used to map the type string,
# to the label index we used in the UML diagram, used later to link entities
message_mapping = {}

entry_index = 2
for _type, message in types.items():
    message_mapping[_type] = entry_index
    entry_index += 1

print(message_mapping)

{'PbMeta': 2, 'PbTimeZone': 3, 'PbStateTotals': 4, 'PbWorkflow': 5, 'PbJob': 6, 'PbTask': 7, 'PbPollTask': 8, 'PbCondition': 9, 'PbPrerequisite': 10, 'PbTaskProxy': 11, 'PbFamily': 12, 'PbFamilyProxy': 13, 'PbEdge': 14, 'PbEdges': 15, 'PbEntireWorkflow': 16}


In [4]:
# Generate dot syntax to display a class diagram
# Refs:
# - https://fsteeg.wordpress.com/2006/11/17/uml-class-diagrams-with-graphviz/
# - http://www.ffnn.nl/pages/articles/media/uml-diagrams-using-graphviz-dot.php

template = StringIO()

template.write(f"""
digraph "Protobuf UML class diagram" {{
    fontname = "Bitstream Vera Sans"
    fontsize = 8

    node [
        fontname = "Bitstream Vera Sans"
        fontsize = 8
        shape = "record"
        style=filled
        fillcolor=gray95
    ]

    edge [
        fontname = "Bitstream Vera Sans"
        fontsize = 8

    ]

""")

relationships = []

entry_index = 2
for _type, message in types.items():
    type_template_text = StringIO()
    type_template_text.write(f"""    {entry_index}[label = "{{{_type}|""")  #]\n""")
    fields = []
    for _field in message.fields:
        message_type = _field.message_type
        field_type = type_mapping[_field.type] # this will be 'message' if referencing another protobuf message type
        
        if message_type:
            this_node = message_mapping[_type]
            that_node = message_mapping[message_type.name]
            relationships.append(f"{this_node}->{that_node}")
            field_type = message_type.name # so we replace the 'message' token by the actual name
        
        fields.append(f"+ {_field.name}:{field_type}")
    
    # add fields
    type_template_text.write("\\n".join(fields))
    type_template_text.write("}\"]\n")
    entry_index += 1
    template.write(type_template_text.getvalue())
    
    type_template_text.close()

# add relationships
template.write("\n")
template.write("\n".join(relationships))

template.write("\n}")
template_text = template.getvalue()
template.close()

print(template_text)


digraph "Protobuf UML class diagram" {
    fontname = "Bitstream Vera Sans"
    fontsize = 8

    node [
        fontname = "Bitstream Vera Sans"
        fontsize = 8
        shape = "record"
        style=filled
        fillcolor=gray95
    ]

    edge [
        fontname = "Bitstream Vera Sans"
        fontsize = 8

    ]

    2[label = "{PbMeta|+ title:string\n+ description:string\n+ URL:string\n+ user_defined:string}"]
    3[label = "{PbTimeZone|+ hours:int32\n+ minutes:int32\n+ string_basic:string\n+ string_extended:string}"]
    4[label = "{PbStateTotals|+ runahead:int32\n+ waiting:int32\n+ held:int32\n+ queued:int32\n+ expired:int32\n+ ready:int32\n+ submit_failed:int32\n+ submit_retrying:int32\n+ submitted:int32\n+ retrying:int32\n+ running:int32\n+ failed:int32\n+ succeeded:int32}"]
    5[label = "{PbWorkflow|+ checksum:string\n+ id:string\n+ name:string\n+ status:string\n+ host:string\n+ port:int32\n+ owner:string\n+ tasks:string\n+ families:string\n+ edges:PbEdges\n+ api_ver

In [5]:
from graphviz import Source

src = Source(template_text)
src
src.render('/tmp/uml')

'/tmp/uml.pdf'